## Run the next two cells to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [1]:
from ipywidgets import interact
import ipywidgets as widgets
from analysis_logger import main
from measurement_directory import *
# main('zcam_dual_imaging', r'08\200814\run1_test')

def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['zcam_dual_imaging', 'fake analysis' , 'fake analysis 2', 'ycam'])
measurement_menu = widgets.Dropdown(options = todays_measurements(),
                                   description = 'today\'s measurements' )
load_measurement_button = widgets.Button(description = 'select measurement')
def load_measurement(button):
    global watchfolder
    watchfolder = measurement_directory(measurement_name = measurement_menu.value)
    print(watchfolder)
load_measurement_button.on_click(load_measurement)
widgets.HBox([measurement_menu, load_measurement_button])

interactive(children=(Dropdown(description='analysis_mode', options=('zcam_dual_imaging', 'fake analysis', 'fa…

08\200814\run1_test


In [2]:
try:
    main(analysis_type, watchfolder)
except KeyboardInterrupt:
    pass
except:
    enrico_bot.post_message('analysis crashed.')
    pass



 Watching this folder for changes: 08\200814\run1_test


loading matlab engine...


DEBUG:analysis_logger:223736_0.spe analyzing: 


matlab engine loaded
K_bareNcntAverageMarqueeBoxValues 10.256646889570137
Na_bareNcntAverageMarqueeBoxValues 527.984158155121
Na_COMX 115.43487068027882
Na_COMY 38.91158646148183


DEBUG:analysis_logger:223735_0.spe analyzing: 


run_id 223736 analyzed: {'K_bareNcntAverageMarqueeBoxValues': 10.256646889570137, 'Na_bareNcntAverageMarqueeBoxValues': 527.984158155121, 'Na_COMX': 115.43487068027882, 'Na_COMY': 38.91158646148183}

FBinGauss: 
79.27
repetition: 
2.0
PAtime: 
10.0
AOMPA: 
365.0
PAlogic: 
0.0


K_bareNcntAverageMarqueeBoxValues 16.6359384849466
Na_bareNcntAverageMarqueeBoxValues 372.59194597814155
Na_COMX 115.51904616941157
Na_COMY 39.91813385010637


...

DEBUG:analysis_logger:223741_0.spe analyzing: 


K_bareNcntAverageMarqueeBoxValues 4.146152907329437
Na_bareNcntAverageMarqueeBoxValues 494.4099293788605
Na_COMX 118.09111483958964
Na_COMY 39.538719285977585
run_id 223741 analyzed: {'K_bareNcntAverageMarqueeBoxValues': 4.146152907329437, 'Na_bareNcntAverageMarqueeBoxValues': 494.4099293788605, 'Na_COMX': 118.09111483958964, 'Na_COMY': 39.538719285977585}

FBinGauss: 
79.27
repetition: 
2.0
PAtime: 
10.0
AOMPA: 
395.0
PAlogic: 
1.0




DEBUG:analysis_logger:223742_0.spe analyzing: 


K_bareNcntAverageMarqueeBoxValues 5.433165021980102
Na_bareNcntAverageMarqueeBoxValues 253.38338152281065
Na_COMX 116.676492470557
Na_COMY 39.39863303896362
run_id 223742 analyzed: {'K_bareNcntAverageMarqueeBoxValues': 5.433165021980102, 'Na_bareNcntAverageMarqueeBoxValues': 253.38338152281065, 'Na_COMX': 116.676492470557, 'Na_COMY': 39.39863303896362}

FBinGauss: 
79.27
repetition: 
2.0
PAtime: 
10.0
AOMPA: 
395.0
PAlogic: 
0.0




KeyboardInterrupt: 

In [1]:
# breadboard_repo_path =  r'D:\Fermidata1\enrico\breadboard-python-client\\'
#loading takes a while, don't run this if testing
# print('loading matlab engine...')
# import matlab.engine
# eng = matlab.engine.start_matlab()
# print('matlab engine loaded')

# import ipywidgets as widgets
# from ipywidgets import interact
# from measurement_directory import *
# import sys
# sys.path.insert(0, breadboard_repo_path)
# from breadboard import BreadboardClient
# bc = BreadboardClient(config_path='API_CONFIG_fermi1.json') # enter your path to the API_config
# import os
# import time
# import datetime
# import shutil
# import warnings
# import enrico_bot
# from image_watchdog import getFileList

# # import pandas as pd
# # from numpy import isnan
# # analysis_type = 'fake analysis'

# # define analysis_function which accepts image filepath and returns dictionary of {analyzed_var_name:analysis_value}
# if analysis_type == 'fake analysis':
#     from random import randint
#     def analysis_function(filepath):
#         time.sleep(randint(0,2))
#         return {'fake analysis 1': randint(0,42), 'fake analysis 2': randint(0,42)}
#     analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
# elif analysis_type == 'fake analysis 2':
#     from random import randint
#     def analysis_function(filepath):
#         time.sleep(randint(0,5))
#         return {'fake analysis 3': randint(0,42), 'fake analysis 4': randint(0,42)}
#     analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
# elif analysis_type == 'ycam':
#     analyzed_var_names = ['bareNcntAverageMarqueeBoxValues', 
#                           'COMX', 'COMY'] #TODO set MATLAB analyzed_var_names properly
#     from matlab_wrapper import getYcamAnalysis
#     def analysis_function(filepath, previous_settings = None):
#         if previous_settings is None:
#             matlab_dict = getYcamAnalysis(eng, filepath)
#         else:
# #             print('previous settings: ' + previous_settings)
# #             return {'fakematlab1': randint(0,42), 'fakematlab2': randint(0,42)}, 'foo1'
#             matlab_dict = getYcamAnalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
#                                            normBox = previous_settings['normBox'])
#         analysis_dict, settings = matlab_dict['analysis'], matlab_dict['settings']
#         return analysis_dict, settings
# elif analysis_type == 'zcam_dual_imaging':
#     analyzed_var_names = ['K_bareNcntAverageMarqueeBoxValues', 'Na_bareNcntAverageMarqueeBoxValues',
#                           'Na_COMX', 'Na_COMY'] #TODO set MATLAB analyzed_var_names properly
#     from dual_imaging_wrapper import getDualImagingAnalysis
#     def analysis_function(filepath, previous_settings = None):
#         if previous_settings is None:
#             matlab_dict = getDualImagingAnalysis(eng, filepath)
# #         else:
# #             matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
# #                                            normBox = previous_settings['normBox'])
#         analysis_dict = matlab_dict['analysis']
#         settings = None
#         return analysis_dict, settings 

# warnings.filterwarnings(
#     "ignore", "Your application has authenticated using end user credentials")
# warnings.filterwarnings(
#     "ignore", "Could not find appropriate MS Visual C Runtime")
# bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

# def analyze_image(image_filename, previous_settings = None, output_previous_settings = True):
#     #TODO adapt for triple imaging later
#     abs_image_path = os.path.join(os.path.join(os.getcwd(), watchfolder), image_filename)
#     settings = None
#     print('{file} analyzing: '.format(file=image_filename))
#     if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
#         analysis_dict = analysis_function(image_filename)
#     elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging') and previous_settings is None:
#         analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
#     elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging'):
#         analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
#     if not output_previous_settings:
#         settings = None #forces user to select new marquee box for each shot
#     for key in analyzed_var_names:
#         print(key, analysis_dict[key])
#     return analysis_dict, settings

# def main():
#     # Global settings
#     refresh_time = 1  # seconds
#     previous_settings = None
#     print("\n\n Watching this folder for changes: " + watchfolder + "\n\n")
#     unanalyzed_files = []
#     done_files = []
#     append_mode = True

#     # Main Loop
#     while True:
#         if not os.path.exists(watchfolder):
#             time.sleep(refresh_time)
#             continue
#         else:
#             files, _ = getFileList(watchfolder)
#             fresh_files = sorted(list(set(files).difference(set(done_files)).difference(set(unanalyzed_files))))
#             unanalyzed_files += fresh_files #push newest files to top of stack
        
#         for file in reversed(unanalyzed_files): #start from top of stack
#             run_id = run_id_from_filename(file)
#             if append_mode:
#                 run_dict = bc._send_message('get', '/runs/' + str(run_id) + '/').json()
#                 if set(analyzed_var_names).issubset(set(run_dict.keys())):
#                     popped_file = [unanalyzed_files.pop()]
#                     done_files += popped_file
#                     continue
#             analysis_dict, previous_settings = analyze_image(file, previous_settings)
#             resp = bc.append_analysis_to_run(run_id, analysis_dict)
#             popped_file = [unanalyzed_files.pop()]
#             done_files += popped_file
#             print('\n')

#         time.sleep(refresh_time)
# # try:
# #     main()
# # except:
# #     enrico_bot.post_message('analysis crashed, restart it.')
# #     pass


# def set_analysis_type(analysis_mode):
#     global analysis_type
#     analysis_type = analysis_mode
#     print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

# mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'ycam', 'zcam_dual_imaging'])
# measurement_menu = widgets.Dropdown(options = todays_measurements(),
#                                    description = 'today\'s measurements' )
# load_measurement_button = widgets.Button(description = 'select measurement')
# def load_measurement(button):
#     global watchfolder
#     watchfolder = measurement_directory(measurement_name = measurement_menu.value)
#     print(watchfolder)
#     main()
# load_measurement_button.on_click(load_measurement)
# widgets.HBox([measurement_menu, load_measurement_button])

NameError: name 'analysis_type' is not defined